In [1]:
import keras
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Input,ZeroPadding2D,InputLayer
from keras.models import Model

C:\Users\pujit\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_data=pd.read_csv("Train.csv")
target=train_data['label']
train_data.drop('label', inplace=True, axis=1)
np.shape(train_data)

(11995, 784)

In [3]:
np.shape(target)

(11995,)

In [4]:
test_data=pd.read_csv("Test.csv")
np.shape(test_data)

(3000, 784)

In [5]:
train_data=np.array(train_data)
test_data=np.array(test_data)
target=np.array(target)

In [6]:
train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

In [7]:
np.shape(train_data)

(11995, 28, 28, 1)

In [8]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data /= 255
test_data /= 255
print('x_train shape:', train_data.shape)
print(train_data.shape[0], 'train samples')
print(test_data.shape[0], 'test samples')

x_train shape: (11995, 28, 28, 1)
11995 train samples
3000 test samples


In [9]:
target = keras.utils.to_categorical(target, 10)
np.shape(target)

(11995, 10)

In [10]:
model=Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", 
          input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dense(128))
model.add(LeakyReLU())

model.add(Dense(10, activation='softmax'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 256)         0         
__________

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [14]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

In [15]:
train_generator = gen.flow(train_data, target, batch_size=128)

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=11995//128, epochs=47)

In [ ]:
pred=model.predict(x=test_data)
pred=np.argmax(a=pred,axis=1)
np.shape(pred)
pred=pd.DataFrame(pred)
pred.to_csv("predictions.csv")